In [1]:
import pandas as pd

NameCol = 'Name'
DisplayCol = 'Name (to be displayed on doorcard)'
YearCol = 'Year'
MajorCol = 'Major'
CaptionCol = 'Doorcard Caption'
ExcelLocation = './doorcards.xlsx'

TemplateLocation = './templates/URSA_Door_card.pptx'
LayoutName = 'Doorcard'
FontLocation = './font/DINCondensed-Regular.ttf'

PhotoLocation = './doorcard_photos'
PngDestination = './doorcards_png'
PptxDestination = './doorcards_pptx'


df = pd.read_excel(ExcelLocation)

In [2]:
import re
import os
from PIL import Image
import aspose.slides as slides
import aspose.pydrawing as drawing

def ProcessField(s):
    return re.sub(r'(?<!\w)and(?!\w)',"&", s).upper()

def GetFileName(name):
    name = name[:20].strip()
    picArr = os.listdir(PhotoLocation)
    filename = []
    for x in picArr:
        searched = re.search(name, x)
        if searched:
            filename.append(x)
    if len(filename) == 0:
        print("No picture found for {0}".format(name))
        return None
    if len(filename) > 1:
        print("Ambiguity in filename for {0}. Choosing first path".format(name), filename)
    return filename[0]

def PrimePics():
    imagenames = os.listdir(PhotoLocation)
    for imagename in imagenames:
        path = os.path.join(PhotoLocation, imagename)
        try:
            im = Image.open(path)
            im.save(path)
        except Exception as e:
            print(e)

In [3]:
from logging import PlaceHolder
from pptx import Presentation
from pptx.util import Pt
import os
import re
from PIL import Image

def CreateDoorcard(name, data_dict):
    prs = Presentation(TemplateLocation)
    phs = prs.slides[0].placeholders
    for ph in phs:
        if ph.name == "Picture":
            ph.insert_picture(os.path.join(PhotoLocation, GetFileName(name)))
        else:
            ph.text = ProcessField(data_dict[ph.name])

    safeName = re.sub(r'[^A-z]', "", name) + "_Ursa.pptx"
    if not os.path.exists(PptxDestination):
        os.makedirs(PptxDestination)
    prs.save(os.path.join(PptxDestination, safeName))


## Experimental. Doesnt work
def BuildDoorcards(prs, name, data_dict):
    refSlide = Presentation(TemplateLocation).slides[0]
    if prs == None:
        prs = Presentation()
    sl = prs.slides.add_slide(refSlide.slide_layout)
    sl.name = re.sub(r'[^A-z]', "", name) + "_Ursa"

    for ph, refph in zip(list(sl.placeholders), list(refSlide.placeholders)):
        print(refph.name, ph.name)
        if refph.name == "Picture":
            ph.insert_picture(os.path.join(PhotoLocation, GetFileName(name)))
        else:
            ph.text = ProcessField(data_dict[refph.name])
    return prs

In [4]:
PrimePics()
for i, row in df.iterrows():
    try:
        CreateDoorcard(row[NameCol], {"Name": row[DisplayCol], "Year": row[YearCol], "Major": row[MajorCol], "Caption": row[CaptionCol]})
    except Exception as e:
        print("Error at {0}".format(row[NameCol]), e)

Ambiguity in filename for Min-Rei Seah. Choosing first path ['DA12265F-53A6-4947-A972-B886BE687E60_Min-Rei Seah.jpeg', 'photo1659344868_Min-Rei Seah.jpeg']
Ambiguity in filename for Min-Rei Seah. Choosing first path ['DA12265F-53A6-4947-A972-B886BE687E60_Min-Rei Seah.jpeg', 'photo1659344868_Min-Rei Seah.jpeg']
